# Data-X Spring 2019: Homework 7

### Webscraping



In this homework, you will do some exercises with web-scraping.

## Name: Tirth Patel

## SID: 3034227694



### Fun with Webscraping & Text manipulation


## 1. Statistics in Presidential Debates

Your first task is to scrape Presidential Debates from the Commission of Presidential Debates website: https://www.debates.org/voter-education/debate-transcripts/

To do this, you are not allowed to manually look up the URLs that you need, instead you have to scrape them. The root url to be scraped is the one listed above, namely: https://www.debates.org/voter-education/debate-transcripts/


1. By using `requests` and `BeautifulSoup` find all the links / URLs on the website that links to transcriptions of **First Presidential Debates** from the years [1988, 1984, 1976, 1960]. In total you should find 4 links / URLs that fulfill this criteria. **Print the urls.**

2. When you have a list of the URLs your task is to create a Data Frame with some statistics (see example of output below):
    1. Scrape the title of each link and use that as the column name in your Data Frame. 
    2. Count how long the transcript of the debate is (as in the number of characters in transcription string). Feel free to include `\` characters in your count, but remove any breakline characters, i.e. `\n`. You will get credit if your count is +/- 10% from our result.
    3. Count how many times the word **war** was used in the different debates. Note that you have to convert the text in a smart way (to not count the word **warranty** for example, but counting **war.**, **war!**, **war,** or **War** etc.
    4. Also scrape the most common used word in the debate, and write how many times it was used. Note that you have to use the same strategy as in C in order to do this.
    
    **Print your final output result.**
    
**Tips:**

___

In order to solve the questions above, it can be useful to work with Regular Expressions and explore methods on strings like `.strip(), .replace(), .find(), .count(), .lower()` etc. Both are very powerful tools to do string processing in Python. To count common words for example I used a `Counter` object and a Regular expression pattern for only words, see example:

```python
    from collections import Counter
    import re

    counts = Counter(re.findall(r"[\w']+", text.lower()))
```

Read more about Regular Expressions here: https://docs.python.org/3/howto/regex.html
    
    
**Example output of all of the answers to Question 1.2:**


![pres_stats_2](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/presidents_stats_2.jpg)



----

.




In [2]:
# your code here
import requests
import bs4 as bs

In [3]:
source = requests.get('https://www.debates.org/voter-education/debate-transcripts/')
soup = bs.BeautifulSoup(source.content, features = 'html.parser')

In [4]:
links = soup.find_all('a')
text_list = []# store text of a link temporarily
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
years = [1988, 1984, 1976, 1960]
for link in links:
    if link.text != '':
        text = link.text.split()
        sorted_text = sorted(text, reverse = True)
        if (sorted_text[0] == 'Transcript'): # extracting all links which contain the word 'Transcript'
            text_list.append(text)

In [5]:
start = 0; # to determine the start index while searching for years
final_links = [] # this list contains the relevant text corresponding to links
for year in years:
        dates_in_year = []
        
        for i in range(start, len(text_list)):
            if int(text_list[i][2]) == year:
                temp = text_list[i] # possible candidate for link
                monthcode =  months.index(text_list[i][0]) # month in numbers
                date = int(text_list[i][1][:-1]) # removing last char (,) from all dates
                dates_in_year.append((monthcode, date, text_list[i]))
                start = i; # next time searching can start from this point (don't do this if years are not in a particular order)
                
        final_links.append(' '.join(sorted(dates_in_year)[0][2]))# appending the text corresponding to earliest debate for that year        

print(final_links)   

['September 25, 1988 Debate Transcript', 'October 7, 1984 Debate Transcript', 'September 23, 1976 Debate Transcript', 'September 26, 1960 Debate Transcript']


In [6]:
#getting the URLs corresponding to texts obtained above
urls = []

for link in links:
   
    for final_link in final_links:
        if str.strip(link.text) == final_link: # you need to strip link.text as it contains leading spaces
            urls.append('https://www.debates.org'+link.get('href'))
            
print(urls)         

['https://www.debates.org/voter-education/debate-transcripts/september-25-1988-debate-transcript/', 'https://www.debates.org/voter-education/debate-transcripts/october-7-1984-debate-transcript/', 'https://www.debates.org/voter-education/debate-transcripts/september-23-1976-debate-transcript/', 'https://www.debates.org/voter-education/debate-transcripts/september-26-1960-debate-transcript/']


In [7]:
import pandas as pd

In [8]:
column_names = []
for url in urls:
    source = requests.get(url)
    soup = bs.BeautifulSoup(source.content, 'html.parser')
    column_names.append(soup.find('title').text)
    
#print(column_names)    

In [9]:
df = pd.DataFrame(columns = column_names)

In [10]:
from collections import Counter
import re

In [16]:
char_lengths = []
war_lengths = []
common_words = []
common_lengths = []
r = 0
c = 0
for url in urls:
    #print(url)
    count = 0
    count_n = 0 
    source = requests.get(url)
    soup = bs.BeautifulSoup(source.content, 'html.parser')
    body = soup.find_all('body')
    p_tags = soup.find_all('p')
    total_tags = ''
    for i in range(1,len(p_tags)): # the first p tag contains the entire body for some reason, so it is ignored
        p = p_tags[i]
        total_tags = total_tags + str(p)
        #print(p)
        count_n = count_n + max(p.text.count('\n'),0)
        count = count + len(p.text)
    char_lengths.append(count - count_n)
    counts = dict(Counter(re.findall(r"[\w']+", total_tags.lower())))# dict storing words as keys and frequencies as values
    #print(counts.keys())
    war_lengths.append(counts['war'])
    max_key, max_value = max(counts.items(), key=lambda x:x[1])
    common_words.append(max_key)
    common_lengths.append(max_value)

In [17]:
df.loc['Debate char length'] = char_lengths
df.loc['war_count'] = war_lengths
df.loc['most_common_w'] = common_words
df.loc['common_lengths'] = common_lengths

In [18]:
df

,"CPD: September 25, 1988 Debate Transcript","CPD: October 7, 1984 Debate Transcript","CPD: September 23, 1976 Debate Transcript","CPD: September 26, 1960 Debate Transcript"
Debate char length,87511,86534,80758,60960
war_count,8,2,7,3
most_common_w,the,the,the,the
common_lengths,805,868,858,780


    
## 2. Download and read in specific line from many data sets

Scrape the first 27 data sets from this URL http://people.sc.fsu.edu/~jburkardt/datasets/regression/ (i.e.`x01.txt` - `x27.txt`). Then, save the 5th line in each data set, this should be the name of the data set author (get rid of the `#` symbol, the white spaces and the comma at the end). 

Count how many times (with a Python function) each author is the reference for one of the 27 data sets. Showcase your results, sorted, with the most common author name first and how many times he appeared in data sets. Use a Pandas DataFrame to show your results, see example. **Print your final output result.**

**Example output of the answer for Question 2:**

![author_stats](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/data_authors.png)


In [19]:
# your code here
author_dict = {}

In [20]:
# your code here
url2 = 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/'
source = requests.get(url2)
soup = bs.BeautifulSoup(source.content, 'html.parser')
tags = soup.find_all('a')
for i, tag in enumerate(tags):# finding position of first dataset
    if tag.text[:3] == 'x01':
        position = i        
tags = tags[position:position+27]
for tag in tags:
    actual_tag = 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/' + str(tag.text)
    #print(actual_tag)
    source = requests.get(actual_tag)
    soup = bs.BeautifulSoup(source.content, 'html')
    p_tags = soup.find_all('p')
    temp = p_tags[0].text.splitlines()[4] #getting the 5th line
    temp_2 = temp[1:-1].strip()# removing # and whitespaces
    if temp_2 in author_dict:
        author_dict[temp_2] += 1
    else:
        author_dict[temp_2] = 1

In [21]:
df_2 = pd.DataFrame(columns = ['Authors', 'Counts'])

In [22]:
for key, value in author_dict.items():
    df_2.loc[key] = value

In [23]:
df_2 = df_2.sort_values(df_2.columns[1], ascending = False)

In [24]:
df_2 = df_2.drop('Authors', axis = 1)

In [25]:
df3 = df_2.reset_index()
df3.columns = ['Authors', 'Counts']

In [26]:
df3

,Authors,Counts
0,Helmut Spaeth,16
1,"S Chatterjee, B Price",3
2,R J Freund and P D Minton,2
3,D G Kleinbaum and L L Kupper,2
4,"S C Narula, J F Wellington",2
5,K A Brownlee,1
6,S Chatterjee and B Price,1
